In [ ]:
pip install contractions cleantext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 8.4 MB/s eta 0:00:00


In [ ]:
import contractions
import matplotlib.pyplot as plt
import string
import re
from cleantext import clean
import random
from bs4 import BeautifulSoup
import requests



class Preprocessing_token():
    def __init__(self,token):
        self.token = token

    # returns the expanded version of contractions
    def remove_contractions(self, token):
        token = contractions.fix(token.lower())
        return token
    
    #convert all words to lower case
    def remove_uppercase(self, token):
        token = token.lower()
        return token
    
    #Remove Punctuation
    def remove_punctuation(self, token):
        token =  re.sub('[%s]' % re.escape(string.punctuation), '' , token)
        return token
    
    #Remove Numbers
    def remove_numbers(self, token):
        token = re.sub(r'\d+', '', token)
        return token
    
    #Remove whitespace
    def remove_whitespace(self, token):
        token = " ".join(token.split()) #split text then join with space between words
        return token
    
    #remove Emojis 
    def remove_emojis(self, token):
        regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
        return regrex_pattern.sub(r'',token)

    #remove html tags based on given filter tags (it basically gets the content inside given filter tags)
    def remove_html(self,html,tags):
      #using html parser to sort out text only 
      soup = BeautifulSoup(html, 'html.parser')
      #scraping only title and paragraph
      results = soup.find_all(tags)
      #saving the results generated
      text = [result.text for result in results]
      ARTICLE = ' '.join(text)
      return ARTICLE

    

In [ ]:
URL = "https://www.sondakika.com/politika/haber-cumhurbaskani-erdogan-eve-donus-projeleri-ile-6-bin-bilim-insanimizi-geri-kazandirdik-15822485/"
r = requests.get(URL)

pre = Preprocessing_token("")
print(pre.remove_html(r.text,["h1","p"]))


Cumhurbaşkanı Erdoğan: 6 bin bilim insanımızı milletimize, memleketimize geri kazandırdık Cumhurbaşkanı Recep Tayyip Erdoğan, sosyal medya üzerinden dikkat çeken bir paylaşıma imza attı. 2018 yılında hayata geçirilen Eve Dönüş Projeleri kapsamında ülkeye geri dönen bilim insanlarının sayısını veren Erdoğan, " Türkiye, artık gidenlerin geldiği bir ülke" dedi.  Erdoğan paylaşımına şu notu düştü:   "2018'de hayata geçirdiğimiz Eve Dönüş Projeleri kapsamında yabancı ülkelerde faaliyet gösteren 6 bin bilim insanımızı milletimize, memleketimize geri kazandırdık. Beyin göçünü tersine çevirmeye devam edeceğiz. Çünkü Türkiye, artık gidenlerin geldiği bir ülke"  Paylaşımda, ilköğrenimini Muş'ta tamamlayan ve yıllar sonra İngiltere'de ders vermeye başlayan bir bilim insanının, Erdoğan'ın yazdığı mektupla ülkeye geri dönüş hikayesinin anlatıldığı video da yer aldı. Videoda, "Bu ülke birçok cevherini imkansızlıklar yüzünden kaybetti. Artık buna bir son verip yarım kalan hayallerimizi tamamlama zama

In [ ]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import Counter
from heapq import nlargest

nlp = spacy.load('en_core_web_sm')
doc ="""Machine learning (ML) is the scientific study of algorithms and statistical models that computer systems use to progressively improve their performance on a specific task. Machine learning algorithms build a mathematical model of sample data, known as "training data", in order to make predictions or decisions without being explicitly programmed to perform the task. Machine learning algorithms are used in the applications of email filtering, detection of network intruders, and computer vision, where it is infeasible to develop an algorithm of specific instructions for performing the task. Machine learning is closely related to computational statistics, which focuses on making predictions using computers. The study of mathematical optimization delivers methods, theory and application domains to the field of machine learning. Data mining is a field of study within machine learning, and focuses on exploratory data analysis through unsupervised learning.In its application across business problems, machine learning is also referred to as predictive analytics."""

#Cleaning
preprocessor = Preprocessing_token("")
doc = preprocessor.remove_uppercase(doc)
#doc = preprocessor.remove_punctuation(doc)
doc = preprocessor.remove_numbers(doc)
doc = preprocessor.remove_whitespace(doc)
doc = preprocessor.remove_emojis(doc)
doc = preprocessor.remove_contractions(doc)

def summarising(doc):
  preprocessor = Preprocessing_token("")
  doc = preprocessor.remove_uppercase(doc)
  doc = preprocessor.remove_punctuation(doc)
  doc = preprocessor.remove_numbers(doc)
  doc = preprocessor.remove_whitespace(doc)
  doc = preprocessor.remove_emojis(doc)
  doc = preprocessor.remove_contractions(doc)
  doc = nlp(doc)

  keyword = []
  stopwords = list(STOP_WORDS)
  pos_tag = ['PROPN', 'ADJ', 'NOUN', 'VERB']
  for token in doc:
      if(token.text in stopwords or token.text in punctuation):
          continue
      if(token.pos_ in pos_tag):
          keyword.append(token.text)
  freq_word = Counter(keyword)
  max_freq=Counter(keyword).most_common(1)[0][1]
  print(max_freq)
  for word in freq_word.keys():  
          freq_word[word] = (freq_word[word]/max_freq)
  sent_strength={}
  for sent in doc.sents:
      for word in sent:
          if word.text in freq_word.keys():
              if sent in sent_strength.keys():
                  sent_strength[sent]+=freq_word[word.text]
              else:
                  sent_strength[sent]=freq_word[word.text]

  #Set the output summarized sentence count
  summarized_sentences = nlargest(3, sent_strength, key=sent_strength.get)


  final_sentences = [ w.text for w in summarized_sentences ]
  summary = ' '.join(final_sentences)

  return summary

URL = "https://www.sondakika.com/politika/haber-cumhurbaskani-erdogan-eve-donus-projeleri-ile-6-bin-bilim-insanimizi-geri-kazandirdik-15822485/"
r = requests.get(URL)
pre = Preprocessing_token("")
azza = pre.remove_html(r.text,["h1","p"])


some_equal= summarising(azza)
print(some_equal)



7
cumhurbaşkanı erdoğan bin bilim insanımızı milletimize memleketimize geri kazandırdık cumhurbaşkanı recep tayyip erdoğan sosyal medya üzerinden dikkat çeken bir paylaşıma imza attı yılında hayata geçirilen eve dönüş projeleri kapsamında ülkeye geri dönen bilim insanlarının sayısını veren erdoğan türkiye artık gidenlerin geldiği bir ülke dedi erdoğan paylaşımına şyou notu düştü de hayata geçirdiğimiz eve dönüş projeleri kapsamında yabancı ülkelerde faaliyet gösteren bin bilim insanımızı milletimize memleketimize geri kazandırdık beyin göçünü tersine çevirmeye devam edeceğiz çünkü türkiye artık gidenlerin geldiği bir ülke paylaşımda ilköğrenimini muşta tamamlayan ve yıllar sonra i̇ngilterede ders vermeye başlayan bir bilim insanının erdoğanın yazdığı mektupla ülkeye geri dönüş hikayesinin anlatıldığı video da yer aldı videoda bu ülke birçok cevherini imkansızlıklar yüzünden kaybetti artık buna bir son verip yarım kalan hayallerimizi tamamlama zamanı yılında hayata geçirdiğimiz eve dönü